In [1]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
local_filename = 'input.txt'

with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192): 
            f.write(chunk)

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [5]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [6]:
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [23]:
len(data)
torch.randint(2, (10,))

tensor([1, 1, 0, 0, 1, 1, 0, 1, 1, 0])

In [7]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [25]:
ix = torch.randint(len(data) - block_size, (4,))
print(ix)
[data[i+1:i+block_size+1] for i in ix]

tensor([581708,  60900, 214571, 321027])


[tensor([58, 53, 53, 54,  0, 13, 52, 42]),
 tensor([24, 13, 26, 33, 31, 10,  0, 35]),
 tensor([59, 51, 40, 50, 63,  1, 53, 52]),
 tensor([59, 56,  1, 46, 47, 45, 46, 52])]

In [26]:
ix = torch.randint(len(data) - block_size, (4,))
print(ix)
torch.stack([data[i+1:i+block_size+1] for i in ix])

tensor([475970, 119648, 425979, 935139])


tensor([[58,  1, 58, 46, 43, 56, 43,  1],
        [31, 10,  0, 35, 46, 39, 58,  1],
        [ 1, 51, 63,  1, 46, 53, 56, 57],
        [52, 58, 50, 63,  8,  0, 14, 59]])

In [27]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [39]:
xb.shape

torch.Size([4, 8])

In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B,T,C) (batch, time, channel)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:] # (B, C)
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


m = BigramLanguageModel(vocab_size)

logits, loss = m(xb, yb)
print(loss)
    

tensor(4.8786, grad_fn=<NllLossBackward0>)


In [46]:
torch.zeros((4,1), dtype=torch.long)

tensor([[0],
        [0],
        [0],
        [0]])

In [91]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), 400)[0].tolist()))



JUKILingethange,
QUSthe
CLBir: spens omaye tethat t aragle m br. ad.

KIf, hes le word:
Whox whe il ist par DUSIGobe m w Loneyouthyourouthalllow pous:
LII is k.'stheno my. thesemewesil, sicomime Wit reroct
Me mes ud onelour, dueshe fotu appr wigen, dlat se bt;
VI' w's
Findofrd of y orte t f cod orveisthid. moendfanowiecal.
Fa ise me
Thitithenel gl, se whenean! Y:
S: ckin Liechee; m e m:
Y: HAn mo


In [90]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.5484132766723633
